In [5]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os

In [3]:
!pip install rasterio


   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.4 MB 660.6 kB/s eta 0:00:39
   ---------------------------------------- 0.0/25.4 MB 660.6 kB/s eta 0:00:39
   ---------------------------------------- 0.1/25.4 MB 585.1 kB/s eta 0:00:44
   ---------------------------------------- 0.1/25.4 MB 731.4 kB/s eta 0:00:35
   ---------------------------------------- 0.2/25.4 MB 787.7 kB/s eta 0:00:33
   ---------------------------------------- 0.2/25.4 MB 860.2 kB/s eta 0:00:30
    --------------------------------------- 0.4/25.4 MB 1.1 MB/s eta 0:00:24
    --------------------------------------- 0.5/25.4 MB 1.3 MB/s eta 0:00:20
    --------------------------------------- 0.5/25.4 MB 1.3 MB/s eta 0:00:19
    --------------------------------------- 0.6/25.4 MB 1.3 MB/s eta 0:00:20
    --------------------------------------- 0.6/25.4 MB 1.3 MB/s eta 0:00:20
    --------------------------------------- 0.6/25.4 MB 1.3 MB/s eta 0:

In [7]:

gain_file = 'Hansen_GFC-2023-v1.11_gain_40N_080W.tif'  
img_size = 64  
num_classes = 2  
batch_size = 32
epochs = 10
learning_rate = 0.001

In [9]:
print("Checking file...")
if not os.path.exists(gain_file):
    print(f"Error: {gain_file} not found. Place it in the working directory or update the path.")
    exit()
print("File found:", gain_file)

Checking file...
File found: Hansen_GFC-2023-v1.11_gain_40N_080W.tif


In [11]:
print("Loading and preprocessing data...")
with rasterio.open(gain_file) as src:
    gain_data = src.read(1)  
    gain_meta = src.meta
print("Gain data shape:", gain_data.shape)
print("Unique values:", np.unique(gain_data))

Loading and preprocessing data...
Gain data shape: (40000, 40000)
Unique values: [0 1]


In [23]:
patch_size = img_size
stride = patch_size  
images = []
labels = []
height, width = gain_data.shape
for y in tqdm(range(0, height - patch_size + 1, stride)):
    for x in range(0, width - patch_size + 1, stride):
        patch = gain_data[y:y+patch_size, x:x+patch_size]
        if patch.shape[0] != patch_size or patch.shape[1] != patch_size:
            continue  
        patch = patch[:, :, np.newaxis]  
        patch = patch.astype(np.float32) / 1.0  
        images.append(patch)
        label = 1 if np.sum(patch) > 0 else 0  
        labels.append(label)

images = np.array(images)
labels = np.array(labels)
print("Extracted patches:", images.shape)
print("Label distribution (0=No Gain, 1=Gain):", np.bincount(labels))

100%|██████████| 625/625 [00:10<00:00, 59.51it/s]


Extracted patches: (390625, 64, 64, 1)
Label distribution (0=No Gain, 1=Gain): [309760  80865]


In [25]:
if len(images) == 0:
    print("Error: No patches extracted. Check gain data or reduce patch_size.")
    exit()
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels)
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])

Training samples: 312500
Validation samples: 78125


In [27]:
# CNN Model Definition
print("Building CNN model...")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Building CNN model...


D:\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,778 (2.61 MB)

 Trainable params: 683,330 (2.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [29]:
print("Training model...")
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    X_train, y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping],
    verbose=1
)

Training model...
Epoch 1/10
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 800s 81ms/step - accuracy: 0.9962 - loss: 0.0255 - val_accuracy: 0.9979 - val_loss: 0.0149
Epoch 2/10
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 788s 81ms/step - accuracy: 0.9979 - loss: 0.0165 - val_accuracy: 0.2070 - val_loss: 14.6755
Epoch 3/10
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 812s 83ms/step - accuracy: 0.9979 - loss: 0.0158 - val_accuracy: 0.9979 - val_loss: 0.0150
Epoch 4/10
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 809s 83ms/step - accuracy: 0.9977 - loss: 0.0158 - val_accuracy: 0.9979 - val_loss: 0.0149


In [31]:
# Model Evaluation
val_loss, val_acc = model.evaluate(X_val, y_val_cat, verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.savefig('training_history.png')
plt.close()

Validation Accuracy: 0.9979
Validation Loss: 0.0149


In [33]:
# Confusion Matrix
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_val, y_pred_classes)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.close()
print(classification_report(y_val, y_pred_classes, target_names=['No Gain', 'Gain']))

2442/2442 ━━━━━━━━━━━━━━━━━━━━ 26s 11ms/step
              precision    recall  f1-score   support

     No Gain       1.00      1.00      1.00     61952
        Gain       1.00      0.99      0.99     16173

    accuracy                           1.00     78125
   macro avg       1.00      0.99      1.00     78125
weighted avg       1.00      1.00      1.00     78125



In [35]:

print("Performing change detection...")
print("Note: Simulating change detection with patches from the same file. For real change detection, use a loss file or earlier year (e.g., 2022).")
time1_images = X_val[:5]  
time2_images = X_val[5:10]  
time1_pred = model.predict(time1_images)
time1_classes = np.argmax(time1_pred, axis=1)
time2_pred = model.predict(time2_images)
time2_classes = np.argmax(time2_pred, axis=1)
changes = time2_classes - time1_classes  
plt.figure(figsize=(15, 10))
for i in range(5):
    plt.subplot(5, 3, i*3 + 1)
    plt.imshow(time1_images[i][:, :, 0], cmap='Greens')
    plt.title(f"Time 1 ({'No Gain' if time1_classes[i] == 0 else 'Gain'})")
    plt.axis('off')
    plt.subplot(5, 3, i*3 + 2)
    plt.imshow(time2_images[i][:, :, 0], cmap='Greens')
    plt.title(f"Time 2 ({'No Gain' if time2_classes[i] == 0 else 'Gain'})")
    plt.axis('off')
    plt.subplot(5, 3, i*3 + 3)
    if changes[i] == 1:
        plt.imshow(time2_images[i][:, :, 0], cmap='Reds')
        plt.title("Gain Detected!", color='red')
    elif changes[i] == -1:
        plt.imshow(time2_images[i][:, :, 0], cmap='Greens')
        plt.title("Loss of Gain Detected!", color='green')
    else:
        plt.imshow(time2_images[i][:, :, 0], cmap='Greys')
        plt.title("No Significant Change", color='blue')
    plt.axis('off')
plt.tight_layout()
plt.savefig('change_detection.png')
plt.close()


Performing change detection...
Note: Simulating change detection with patches from the same file. For real change detection, use a loss file or earlier year (e.g., 2022).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [37]:
# Visualize Full Gain Map
plt.figure(figsize=(10, 10))
plt.imshow(gain_data, cmap='Greens')
plt.title('Hansen GFC 2023 Gain Map (40N_080W)')
plt.colorbar(label='Gain (1) / No Gain (0)')
plt.savefig('gain_map.png')
plt.close()

In [41]:
model.save('hansen_gain_model.keras')
print("Model saved as 'hansen_gain_model.keras'")

Model saved as 'hansen_gain_model.keras'
